In [4]:
import requests
import json
import csv
import pandas as pd
from datetime import datetime
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googletrans import Translator
from konlpy.tag import Okt
import platform
import sqlite3
from matplotlib import font_manager, rc 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import urllib.parse

NAVER_CLIENT_ID = "GAGg4NiRWNTiGDwteZDu"
NAVER_CLIENT_SECRET = "wcfjuoP_Ej"
DB_FILE = "news_monitoring.db" 

os_name = platform.system()
if os_name == 'Windows':
    font_name = 'Malgun Gothic'
elif os_name == 'Darwin': 
    font_name = 'AppleGothic'
elif os_name == 'Linux':
    font_name = 'NanumGothic'
else:
    font_name = None

if font_name:
    try:
        rc('font', family=font_name)
        print(f"{os_name} OS에서 '{font_name}' 폰트를 설정했습니다.")
    except:
        print(f"'{font_name}' 폰트를 찾을 수 없습니다. 시각화 시 한글이 깨질 수 있습니다.")
else:
    print("지원되지 않는 OS입니다. 폰트 설정이 필요합니다.")


def get_naver_news(query, display=50):
    """네이버 뉴스 API를 호출하는 함수 (최신순 정렬)"""
    encoded_query = urllib.parse.quote(query)
    url = f"https://openapi.naver.com/v1/search/news.json?query={encoded_query}&display={display}&sort=date"
    
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET,
    }
    
    print(f"검색 URL: {url}") 
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    result = response.json()
    print(f"검색 결과 수: {len(result.get('items', []))}개")  
    
    return result.get('items', [])

def test_search_queries(base_query):
    test_queries = [
        base_query,
        
        f"{base_query} 이스라엘",
        f"{base_query} 우크라이나",
        f"{base_query} 가자지구",
        
        f"{base_query} 이스라엘 OR 우크라이나 OR 가자지구",
        
        f"{base_query} 긴급구호",
        f"{base_query} 분쟁",
        
        f"{base_query} 국제",
        f"{base_query} 지원"
    ]
    
    print(f"\n=== '{base_query}' 관련 다양한 검색 패턴 테스트 ===")
    
    results = {}
    for i, query in enumerate(test_queries, 1):
        try:
            print(f"\n[테스트 {i}] 검색어: '{query}'")
            articles = get_naver_news(query, display=50)
            results[query] = len(articles)
            
            if articles:
                print(f"   {len(articles)}개 기사 발견")
                # 첫 번째 기사 제목 출력
                first_title = articles[0].get('title', '').replace('<b>', '').replace('</b>', '')
                print(f"   예시: {first_title[:50]}...")
            else:
                print(f"   검색 결과 없음")
                
        except Exception as e:
            print(f"   오류: {e}")
            results[query] = 0
    
    print(f"\n=== 검색 결과 요약 ===")
    for query, count in results.items():
        print(f"'{query}': {count}개")
    
    return results

def init_db(db_path):
    """데이터베이스 초기화 함수: 'articles' 테이블 생성"""
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS articles (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                search_timestamp TEXT NOT NULL,
                final_query TEXT NOT NULL,
                title TEXT NOT NULL,
                original_link TEXT NOT NULL UNIQUE,
                sentiment_score REAL NOT NULL
            )
        ''')
        conn.commit()

def analyze_and_process_articles(articles, final_query, db_path):
    """뉴스 기사 분석 및 데이터베이스 저장"""
    analyzer = SentimentIntensityAnalyzer()
    translator = Translator()
    okt = Okt()

    total_compound_score = 0
    article_count = 0
    all_descriptions = ""
    new_article_count = 0

    print("\n--- 개별 뉴스 분석 및 결과 저장 ---")
    
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        for article in articles:
            try:
                link = article.get('originallink', '')
                if not link: continue

                cursor.execute("SELECT id FROM articles WHERE original_link = ?", (link,))
                if cursor.fetchone() is not None:
                    continue

                title = article.get('title', '').replace('<b>', '').replace('</b>', '')
                description = article.get('description', '').replace('<b>', '').replace('</b>', '')
                if not description: continue
                
                all_descriptions += description + " "

                translated_text = translator.translate(description, src='ko', dest='en').text
                vs = analyzer.polarity_scores(translated_text)
                compound_score = vs['compound']
                
                print(f" - (신규) 제목: {title}")
                print(f"   감성 점수: {compound_score:.4f}")

                total_compound_score += compound_score
                article_count += 1
                new_article_count += 1

                now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                cursor.execute(
                    "INSERT INTO articles (search_timestamp, final_query, title, original_link, sentiment_score) VALUES (?, ?, ?, ?, ?)",
                    (now, final_query, title, link, compound_score)
                )

            except Exception as e:
                print(f"오류 발생으로 기사 하나를 건너뜁니다: {e}")
                continue
        conn.commit()
    
    print(f"\n>> 총 {new_article_count}개의 새로운 기사를 DB에 저장했습니다.")

    nouns = okt.nouns(all_descriptions)
    filtered_nouns = [n for n in nouns if len(n) > 1]
    
    top_keywords = Counter(filtered_nouns).most_common(10)
    average_score = total_compound_score / article_count if article_count > 0 else 0
    
    return average_score, top_keywords

def visualize_top_keywords_sentiment(db_path):
    """
    DB에 저장된 전체 기사 제목을 분석하여
    상위 10개 키워드의 평균 감성 점수를 비교하는 막대그래프를 생성합니다.
    """
    print("\n[키워드별 감성 분석] DB 전체 데이터를 분석하여 그래프를 생성합니다...")
    
    try:
        with sqlite3.connect(db_path) as conn:
            df = pd.read_sql_query("SELECT title, sentiment_score FROM articles", conn)

        if df.empty:
            print("분석할 데이터가 DB에 없습니다.")
            return

        okt = Okt()
        
        all_titles = ' '.join(df['title'].tolist())
        nouns = okt.nouns(all_titles)
        
        filtered_nouns = [n for n in nouns if len(n) > 1]
        
        top_10_keywords = [keyword for keyword, count in Counter(filtered_nouns).most_common(10)]

        if not top_10_keywords:
            print("분석할 키워드를 찾을 수 없습니다.")
            return

        print(f"\n>> 분석 대상 상위 10개 키워드: {', '.join(top_10_keywords)}")

        keyword_sentiments = {}
        for keyword in top_10_keywords:

            avg_score = df[df['title'].str.contains(keyword, na=False)]['sentiment_score'].mean()
            keyword_sentiments[keyword] = avg_score
            
        sorted_sentiments = sorted(keyword_sentiments.items(), key=lambda item: item[1], reverse=True)
        
        keywords = [item[0] for item in sorted_sentiments]
        scores = [item[1] for item in sorted_sentiments]

        plt.figure(figsize=(12, 8))
        bars = plt.bar(keywords, scores, color='skyblue')
        
        plt.axhline(0, color='gray', linewidth=0.8, linestyle='--')
        
        plt.title('Top 10 Keyword AVG Sentiment Score', fontsize=16)
        plt.xlabel('Keyword', fontsize=12)
        plt.ylabel('AVG Sentiment Score (Neg/Pos)', fontsize=12)
        plt.xticks(rotation=45, ha='right') 
        plt.grid(True, axis='y', linestyle=':', alpha=0.6)
        
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.2f}', va='bottom' if yval >=0 else 'top', ha='center')

        plt.tight_layout() 
        print("\n분석 그래프를 출력합니다. (그래프 창을 닫으면 프로그램이 종료됩니다.)")
        plt.show()

    except Exception as e:
        print(f"그래프 생성 중 오류가 발생했습니다: {e}")


if __name__ == "__main__":
    init_db(DB_FILE)

    base_query = input("검색할 기관/주제명을 입력하세요: ")
    
    print("\n 검색 가능성을 확인하기 위해 다양한 검색 패턴을 테스트합니다...")
    test_results = test_search_queries(base_query)
    
    print("\n[ 검색어 상세화 옵션 - 개선된 버전 ]")
    print("1: 기본 검색어만 사용")
    print("2: 기업 파트너십/CSR 관련 (단순 키워드 조합)")
    print("3: 정책/권리옹호 관련 (단순 키워드 조합)")
    print("4: 국제 분쟁/긴급구호 관련 (개선된 검색)")
    print("5: 기금 마련/캠페인 관련 (단순 키워드 조합)")
    print("6: 테스트에서 가장 결과가 많았던 검색어 사용")

    option = ""
    while option not in ['1', '2', '3', '4', '5', '6']:
        option = input("원하는 옵션을 선택하세요: ")

    if option == '1':
        final_query = base_query
    elif option == '2':
        final_query = f"{base_query} 기업 협약"
    elif option == '3':
        final_query = f"{base_query} 정책 권리"
    elif option == '4':
        conflict_keywords = ["이스라엘", "우크라이나", "가자지구", "긴급구호", "분쟁"]
        
        print(f"\n개별 키워드별 검색 결과:")
        best_keyword = ""
        best_count = 0
        
        for keyword in conflict_keywords:
            test_query = f"{base_query} {keyword}"
            try:
                articles = get_naver_news(test_query, display=5)
                count = len(articles)
                print(f"  '{keyword}': {count}개")
                if count > best_count:
                    best_count = count
                    best_keyword = keyword
            except:
                print(f"  '{keyword}': 오류 발생")
        
        if best_keyword:
            final_query = f"{base_query} {best_keyword}"
            print(f"\n가장 많은 결과를 보인 '{best_keyword}'를 사용합니다.")
        else:
            final_query = f"{base_query} 국제"
            print(f"\n기본값으로 '국제'를 사용합니다.")
    
    elif option == '5':
        final_query = f"{base_query} 캠페인 모금"
    elif option == '6':
        best_query = max(test_results, key=test_results.get)
        if test_results[best_query] > 0:
            final_query = best_query
            print(f"\n테스트에서 가장 좋은 결과({test_results[best_query]}개)를 보인 검색어를 사용합니다.")
        else:
            final_query = base_query
            print(f"\n모든 테스트에서 결과가 없어 기본 검색어를 사용합니다.")

    print(f"\n>> 최종 검색어: '{final_query}'")
    
    try:
        if "YOUR_NAVER_CLIENT_ID" in NAVER_CLIENT_ID:
            raise ValueError("코드에 네이버 클라이언트 ID와 시크릿을 입력해주세요.")
        
        print("네이버 뉴스 API에서 최신순으로 검색 중...")
        news_articles = get_naver_news(final_query)

        if news_articles:
            avg_sentiment, top_keywords = analyze_and_process_articles(news_articles, final_query, DB_FILE)
            
            print("\n" + "="*50)
            print(" 신규 수집 뉴스 분석 결과 요약 ")
            print("="*50)
            print(f" 평균 감성 점수: {avg_sentiment:.4f}")
            if avg_sentiment > 0.05:
                print("  >> 전반적으로 '긍정적' 뉘앙스의 뉴스들이 많습니다.")
            elif avg_sentiment < -0.05:
                print("  >> 전반적으로 '부정적' 뉘앙스의 뉴스들이 많습니다.")
            else:
                print("  >> 전반적으로 '중립적' 뉘앙스의 뉴스들이 많습니다.")
            
            print("\n 주요 핵심 키워드 (상위 10개):")
            for keyword, count in top_keywords:
                print(f"  - {keyword} ({count}회)")
            
            print("\n" + "="*50)
            print(f" 모든 결과를 '{DB_FILE}' 파일에 누적 저장했습니다.")

        else:
            print("새롭게 검색된 뉴스가 없습니다.")
            print("\n 해결 제안:")
            print("1. 검색어를 더 단순하게 만들어보세요")
            print("2. 옵션 6을 선택해 테스트 결과를 활용해보세요")
            print("3. 기본 검색어만으로 다시 시도해보세요")
            
    except Exception as e:
        print(f"\n오류가 발생했습니다: {e}")

    while True:
        choice = input("\nDB 기반 키워드별 감성 분석을 수행하시겠습니까? (y/n): ").lower()
        if choice == 'y':
            visualize_top_keywords_sentiment(DB_FILE)
            break
        elif choice == 'n':
            print("프로그램을 종료합니다.")
            break
        else:
            print("잘못된 입력입니다. 'y' 또는 'n'을 입력해주세요.")

ModuleNotFoundError: No module named 'vaderSentiment'